In [2]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('/home/rrr/ardi/program/pycharm/deploy-streamlit/BankNote_Authentication.csv')
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1372 non-null   float64
 1   skewness  1372 non-null   float64
 2   curtosis  1372 non-null   float64
 3   entropy   1372 non-null   float64
 4   class     1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [5]:
#dependent and independent features
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [10]:
X.head(20)

,variance,skewness,curtosis,entropy
0,3.62160,8.6661,-2.80730,-0.44699
1,4.54590,8.1674,-2.45860,-1.46210
2,3.86600,-2.6383,1.92420,0.10645
3,3.45660,9.5228,-4.01120,-3.59440
4,0.32924,-4.4552,4.57180,-0.98880
5,4.36840,9.6718,-3.96060,-3.16250
6,3.59120,3.0129,0.72888,0.56421
7,2.09220,-6.8100,8.46360,-0.60216
8,3.20320,5.7588,-0.75345,-0.61251
9,1.53560,9.1772,-2.27180,-0.73535


In [11]:
y.head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: class, dtype: int64

In [13]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

((960, 4), (412, 4))

In [15]:
#train random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion)

Accuracy: 0.9951456310679612
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       229
           1       1.00      0.99      0.99       183

    accuracy                           1.00       412
   macro avg       1.00      0.99      1.00       412
weighted avg       1.00      1.00      1.00       412

Confusion Matrix:
 [[229   0]
 [  2 181]]


In [16]:
#save to pickle
import pickle
pickle.dump(rf, open('rf_model.pkl', 'wb'))
model = pickle.load(open('rf_model.pkl', 'rb'))

In [17]:
model.predict([[3.6216, 8.6661, -2.8073, -0.44699]])

/home/rrr/ardi/program/pycharm/deploy-streamlit/.venv/lib64/python3.13/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [ ]:
#analyze model with mlflow, name = third experiment
import mlflow
import mlflow.sklearn
mlflow.set_experiment("third experiment")

mlflow.sklearn.log_model(rf, "random-forest-model")

#record parameters, accuracy, matrix and report
with mlflow.start_run():
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("tn", confusion[0][0])
    mlflow.log_metric("fp", confusion[0][1])
    mlflow.log_metric("fn", confusion[1][0])
    mlflow.log_metric("tp", confusion[1][1])
    mlflow.log_param("model_type", "RandomForestClassifier")    
